In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
%matplotlib inline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.


In [ ]:
train = pd.read_csv("../input/train.csv")
test  = pd.read_csv("../input/test.csv")
gendersub = pd.read_csv("../input/gender_submission.csv")

In [ ]:
train.shape, test.shape, gendersub.shape

EDA

In [ ]:
train.head(5)
test.head(5)

Reading the 4 C's of Data Cleaning from "https://www.kaggle.com/ldfreeman3/a-data-science-framework-to-achieve-99-accuracy", found it useful. Read as followed

"1. Correcting: Reviewing the data, there does not appear to be any aberrant or non-acceptable data inputs. In addition, we see we may have potential outliers in age and fare. However, since they are reasonable values, we will wait until after we complete our exploratory analysis to determine if we should include or exclude from the dataset. It should be noted, that if they were unreasonable values, for example age = 800 instead of 80, then it's probably a safe decision to fix now. However, we want to use caution when we modify data from its original value, because it may be necessary to create an accurate model.
2. Completing: There are null values or missing data in the age, cabin, and embarked field. Missing values can be bad, because some algorithms don't know how-to handle null values and will fail. While others, like decision trees, can handle null values. Thus, it's important to fix before we start modeling, because we will compare and contrast several models. There are two common methods, either delete the record or populate the missing value using a reasonable input. It is not recommended to delete the record, especially a large percentage of records, unless it truly represents an incomplete record. Instead, it's best to impute missing values. A basic methodology for qualitative data is impute using mode. A basic methodology for quantitative data is impute using mean, median, or mean + randomized standard deviation. An intermediate methodology is to use the basic methodology based on specific criteria; like the average age by class or embark port by fare and SES. There are more complex methodologies, however before deploying, it should be compared to the base model to determine if complexity truly adds value. For this dataset, age will be imputed with the median, the cabin attribute will be dropped, and embark will be imputed with mode. Subsequent model iterations may modify this decision to determine if it improves the model’s accuracy.
3. Creating: Feature engineering is when we use existing features to create new features to determine if they provide new signals to predict our outcome. For this dataset, we will create a title feature to determine if it played a role in survival.
4. Converting: Last, but certainly not least, we'll deal with formatting. There are no date or currency formats, but datatype formats. Our categorical data imported as objects, which makes it difficult for mathematical calculations. For this dataset, we will convert object datatypes to categorical dummy variables."

How do we correct data? First we need to examine the data, using 
"print('Train columns with null values:\n', data1.isnull().sum())
print("-"*10)

print('Test/Validation columns with null values:\n', data_val.isnull().sum())
print("-"*10)

data_raw.describe(include = 'all')" - credit to LD Freeman

Run it to test 

In [ ]:
print('Train columns with null values:\n', train.isnull().sum())
print("-"*10)
      
print('Test columns with null values:\n', test.isnull().sum())
print("-"*10)
      
test.describe(include = "all")

In [ ]:
#COMPLETING
test.count(),train.count()

In [ ]:
train.groupby(['Pclass']).mean()
#drop values & replace missing values 
train = train.drop(['PassengerId','Name','Ticket'], axis = 1)
test = test.drop(['Name','Ticket'], axis=1)
test["Age"].fillna(test["Age"].mean(), inplace=True)
train["Age"].fillna(train["Age"].mean(), inplace=True)
test["Fare"].fillna(test["Fare"].mean(), inplace=True)
#because cabin miss too many values, so we drop it directly
train = train.drop(['Cabin'], axis = 1)
test = test.drop(['Cabin'], axis=1)
#because embarked needs a lot of attention, so we will drop it now (i am running out of time)
train = train.drop(['Embarked'], axis = 1)
test = test.drop(['Embarked'], axis=1)
#change sex to 0 and 1
test["Sex"] = test["Sex"].map({"male": 0, "female":1})
train["Sex"] = train["Sex"].map({"male": 0, "female":1})


In [ ]:
train.head()

In [ ]:
#correlation heatmap
corr = test.corr()
sns.heatmap(corr, 
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values)

In [ ]:
#model1 Logistic Regression I found this model on "https://www.kaggle.com/omarelgabry/a-journey-through-titanic")
# define training and testing sets

X_train = train.drop("Survived",axis=1)
Y_train = train["Survived"]
X_test  = test.drop("PassengerId",axis=1).copy()

logreg = LogisticRegression()
logreg.fit(X_train, Y_train)
Y_pred = logreg.predict(X_test)
logreg.score(X_train, Y_train)

In [ ]:
#Model2 Random Forests
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_test)
random_forest.score(X_train, Y_train) 
## maybe overfit

In [ ]:
#print submission
submission = pd.DataFrame({"PassengerId": test["PassengerId"],"Survived": Y_pred})
submission.to_csv("titanichw1.csv",index = False)